<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Classification with Python


Estimated time needed: **25** minutes
    

## Objectives

After completing this lab you will be able to:

* Confidently create classification models


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Let's first load required libraries:


In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing

%matplotlib inline




import warnings
warnings.filterwarnings('ignore')

### About dataset


This dataset is about the performance of basketball teams. The __cbb.csv__ data set includes performance data about five seasons of 354 basketball teams. It includes the following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
|TEAM |	The Division I college basketball school|
|CONF|	The Athletic Conference in which the school participates in (A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference)|
|G|	Number of games played|
|W|	Number of games won|
|ADJOE|	Adjusted Offensive Efficiency (An estimate of the offensive efficiency (points scored per 100 possessions) a team would have against the average Division I defense)|
|ADJDE|	Adjusted Defensive Efficiency (An estimate of the defensive efficiency (points allowed per 100 possessions) a team would have against the average Division I offense)|
|BARTHAG|	Power Rating (Chance of beating an average Division I team)|
|EFG_O|	Effective Field Goal Percentage Shot|
|EFG_D|	Effective Field Goal Percentage Allowed|
|TOR|	Turnover Percentage Allowed (Turnover Rate)|
|TORD|	Turnover Percentage Committed (Steal Rate)|
|ORB|	Offensive Rebound Percentage|
|DRB|	Defensive Rebound Percentage|
|FTR|	Free Throw Rate (How often the given team shoots Free Throws)|
|FTRD|	Free Throw Rate Allowed|
|2P_O|	Two-Point Shooting Percentage|
|2P_D|	Two-Point Shooting Percentage Allowed|
|3P_O|	Three-Point Shooting Percentage|
|3P_D|	Three-Point Shooting Percentage Allowed|
|ADJ_T|	Adjusted Tempo (An estimate of the tempo (possessions per 40 minutes) a team would have against the team that wants to play at an average Division I tempo)|
|WAB|	Wins Above Bubble (The bubble refers to the cut off between making the NCAA March Madness Tournament and not making it)|
|POSTSEASON|	Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)|
|SEED|	Seed in the NCAA March Madness Tournament|
|YEAR|	Season


### Load Data From CSV File  


Let's load the dataset [NB Need to provide link to csv file]


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%206/cbb.csv')
df.head()

In [ ]:
df.shape

## Add Column
Next we'll add a column that will contain "true" if the wins above bubble are over 7 and "false" if not. We'll call this column Win Index or "windex" for short. 


In [ ]:
df['windex'] = np.where(df.WAB > 7, 'True', 'False')

# Data visualization and pre-processing



Next we'll filter the data set to the teams that made the Sweet Sixteen, the Elite Eight, and the Final Four in the post season. We'll also create a new dataframe that will hold the values with the new column.


In [ ]:
df1 = df.loc[df['POSTSEASON'].str.contains('F4|S16|E8', na=False)]
df1.head()

In [ ]:
df1['POSTSEASON'].value_counts()

In [ ]:
#WAB is Wins Above Bubble (The bubble refers to the cut off between making the NCAA March Madness Tournament and not making it)
# WAB ranges from -1.1 and 11.6.  With the decimals I suppose it may be the average wins above bubble in 5 seasons
df1['WAB'].value_counts().sort_index(ascending=False)

In [ ]:
df1.BARTHAG.min(), df1.BARTHAG.max()

32 teams made it into the Sweet Sixteen, 16 into the Elite Eight, and 8 made it into the Final Four over 5 seasons. 


Lets plot some columns to underestand the data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df1.BARTHAG.min(), df1.BARTHAG.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=6)
g.map(plt.hist, 'BARTHAG', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df1.ADJOE.min(), df1.ADJOE.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=2)
g.map(plt.hist, 'ADJOE', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Lets look at how Adjusted Defense Efficiency plots


In [ ]:
bins = np.linspace(df1.ADJDE.min(), df1.ADJDE.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=2)
g.map(plt.hist, 'ADJDE', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

We see that this data point doesn't impact the ability of a team to get into the Final Four. 


## Convert Categorical features to numerical values


Lets look at the postseason:


In [ ]:
df1.groupby(['windex'])['POSTSEASON'].value_counts(normalize=True)

13% of teams with 6 or less wins above bubble make it into the final four while 17% of teams with 7 or more do.


Lets convert wins above bubble (winindex) under 7 to 0 and over 7 to 1:


In [ ]:
df1['windex'].replace(to_replace=['False','True'], value=[0,1],inplace=True)
df1.head()

### Feature selection


Let's define feature sets, X:


In [ ]:
X = df1[['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'windex']]
X[0:5]

What are our lables? Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)|


In [ ]:
y = df1['POSTSEASON'].values
y[0:5]

In [ ]:
#y.aggregate()
#np.count_nonzero(y)
unique_values, counts = np.unique(y, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

## Normalize Data 


Data Standardization gives data zero mean and unit variance (technically should be done after train test split )


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Training and Validation 


Split the data into Training and Validation data.


In [ ]:
# We split the X into train and test to find the best k
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=4)#modified from test_size=0.2 to test_size=0.3 as 
#accuracy and precision for F4 was zero with all max_depth settings
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_val.shape,  y_val.shape)

# Classification 


Now, it is your turn, use the training set to build an accurate model. Then use the validation set  to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



# K Nearest Neighbor(KNN)

<b>Question  1 </b> Build a KNN model using a value of k equals five, find the accuracy on the validation data (X_val and y_val)


You can use <code> accuracy_score</cdoe>


In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [58]:
#Question 1 Answer
neigh = KNeighborsClassifier(n_neighbors = 5).fit(X_train,y_train)
yhat=neigh.predict(X_val)
mean_acc = metrics.accuracy_score(y_val, yhat)
print('acuracy = ',mean_acc)

acuracy =  0.6666666666666666


<b>Question  2</b> Determine and print the accuracy for the first 15 values of k on the validation data:


In [47]:
#Qn 2 Answer
Ks = 16

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_val)
    mean_acc = metrics.accuracy_score(y_val, yhat)
    print('Accuracy with K: ',n,'= ',mean_acc)

Accuracy with K:  1 =  0.3333333333333333
Accuracy with K:  2 =  0.3333333333333333
Accuracy with K:  3 =  0.5
Accuracy with K:  4 =  0.5833333333333334
Accuracy with K:  5 =  0.6666666666666666
Accuracy with K:  6 =  0.5833333333333334
Accuracy with K:  7 =  0.5833333333333334
Accuracy with K:  8 =  0.6666666666666666
Accuracy with K:  9 =  0.5833333333333334
Accuracy with K:  10 =  0.5833333333333334
Accuracy with K:  11 =  0.5833333333333334
Accuracy with K:  12 =  0.5
Accuracy with K:  13 =  0.5833333333333334
Accuracy with K:  14 =  0.5833333333333334
Accuracy with K:  15 =  0.5833333333333334


# Decision Tree


The following lines of code fit a <code>DecisionTreeClassifier</code>:


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Question 3 - Answer
#Based on accuracy it seems min value for max depth is 1

<b>Question  3</b> Determine the minumum   value for the parameter <code>max_depth</code> that improves results 


In [59]:
Ks = 16
mean_acc = np.zeros((Ks-1))
mdepth = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    dtree = DecisionTreeClassifier(criterion="entropy", max_depth = n).fit(X_train,y_train)
    yhat=dtree.predict(X_val)
    mean_acc[n-1] = metrics.accuracy_score(y_val, yhat)
    mdepth[n-1]= n

    
    
    print('when max_depth is: ',n,'accuracy = ',mean_acc[n-1])



when max_depth is:  1 accuracy =  0.6666666666666666
when max_depth is:  2 accuracy =  0.6666666666666666
when max_depth is:  3 accuracy =  0.5
when max_depth is:  4 accuracy =  0.3333333333333333
when max_depth is:  5 accuracy =  0.4166666666666667
when max_depth is:  6 accuracy =  0.5
when max_depth is:  7 accuracy =  0.5
when max_depth is:  8 accuracy =  0.4166666666666667
when max_depth is:  9 accuracy =  0.4166666666666667
when max_depth is:  10 accuracy =  0.4166666666666667
when max_depth is:  11 accuracy =  0.4166666666666667
when max_depth is:  12 accuracy =  0.5
when max_depth is:  13 accuracy =  0.5
when max_depth is:  14 accuracy =  0.4166666666666667
when max_depth is:  15 accuracy =  0.5


# Support Vector Machine


<b>Question  4</b> Train the support  vector machine model and determine the accuracy on the validation data for each kernel. Find the kernel (linear, poly, rbf, sigmoid) that provides the best score on the validation data and train a SVM using it.


#Tried all 4 Kernals below.  Best score was provided by Poly kernal. 

In [51]:
from sklearn import svm

Kernals=['linear', 'poly', 'rbf', 'sigmoid']
Ks = 5
mean_acc = np.zeros((Ks-1))


for n in range(1,Ks):
    
    #Train Model and Predict  
    
    clf = svm.SVC(kernel=Kernals[n-1])
    clf.fit(X_train, y_train) 
    yhat = clf.predict(X_val)
    
    mean_acc[n-1] = metrics.accuracy_score(y_val, yhat)
    print('when kernal is: ',Kernals[n-1],'accuracy = ',mean_acc[n-1])



when kernal is:  linear accuracy =  0.25
when kernal is:  poly accuracy =  0.6666666666666666
when kernal is:  rbf accuracy =  0.5833333333333334
when kernal is:  sigmoid accuracy =  0.5833333333333334


# Logistic Regression


<b>Question 5</b> Train a logistic regression model and determine the accuracy of the validation data (set C=0.01)


###### Question 5 - Answers
Model is trained below and accuracy is reported.  Tried both liblinear and sag solvers.

classification report is created that gives precision, recall, f1score and average accuracy
For the case of teams making it into the final 4 (F4),values are reported below

y       Precision Recall.   f1-score

1       0.50           1.00      0.67

1       0.50           1.00      0.67       

Logloss value is 0.65 for liblinear solver vs 0.35 for sag. Hence Sag solver is recommended for further use

In [39]:
from sklearn.linear_model import LogisticRegression

solvers=['liblinear', 'sag', 'saga', 'lbfgs']
Ks = 5
mean_acc = np.zeros((Ks-1))


for n in range(1,Ks):
    
    #Train Model and Predict  
    LR = LogisticRegression(C=0.01, solver=solvers[n-1])
    LR.fit(X_train, y_train)

    yhat = LR.predict(X_val)
    
    mean_acc[n-1] = metrics.accuracy_score(y_val, yhat)
    print('when solver is: ',solvers[n-1],'accuracy = ',mean_acc[n-1])


when solver is:  liblinear accuracy =  0.5833333333333334
when solver is:  sag accuracy =  0.5833333333333334
when solver is:  saga accuracy =  0.5833333333333334
when solver is:  lbfgs accuracy =  0.5833333333333334


# Model Evaluation using Test set


In [41]:
from sklearn.metrics import f1_score
# for f1_score please set the average parameter to 'micro'
from sklearn.metrics import log_loss

In [42]:
def jaccard_index(predictions, true):
    if (len(predictions) == len(true)):
        intersect = 0;
        for x,y in zip(predictions, true):
            if (x == y):
                intersect += 1
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

<b>Question  5</b> Calculate the  F1 score and Jaccard score for each model from above. Use the Hyperparameter that performed best on the validation data. **For f1_score please set the average parameter to 'micro'.**


### Load Test set for evaluation 


In [43]:
# test_df = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0120ENv3/Dataset/ML0101EN_EDX_skill_up/basketball_train.csv',error_bad_lines=False)
test_df = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0120ENv3/Dataset/ML0101EN_EDX_skill_up/basketball_train.csv')
test_df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Villanova,BE,40,35,123.1,90.9,0.9703,56.1,46.7,16.3,...,30.0,57.4,44.1,36.2,33.9,66.7,8.9,Champions,2.0,2016
2,Notre Dame,ACC,36,24,118.3,103.3,0.8269,54.0,49.5,15.3,...,26.0,52.9,46.5,37.4,36.9,65.5,2.3,E8,6.0,2016
3,Virginia,ACC,37,29,119.9,91.0,0.9600,54.8,48.4,15.1,...,33.4,52.6,46.3,40.3,34.7,61.9,8.6,E8,1.0,2016
4,Kansas,B12,37,32,120.9,90.4,0.9662,55.7,45.1,17.8,...,37.3,52.7,43.4,41.3,32.5,70.1,11.6,E8,1.0,2016


In [ ]:
test_df.shape

In [44]:
test_df['windex'] = np.where(test_df.WAB > 7, 'True', 'False')
test_df1 = test_df[test_df['POSTSEASON'].str.contains('F4|S16|E8', na=False)]
test_Feature = test_df1[['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'windex']]
test_Feature['windex'].replace(to_replace=['False','True'], value=[0,1],inplace=True)
test_X=test_Feature
test_X= preprocessing.StandardScaler().fit(test_X).transform(test_X)
test_X[0:5]

array([[-4.08074446e-01, -1.10135297e+00,  3.37365934e-01,
         2.66479976e+00, -2.46831661e+00,  2.13703245e-01,
         9.44090550e-01, -1.19216365e+00, -1.64348924e+00,
         1.45405982e-02,  1.29523097e+00, -6.23533182e-01,
        -9.31788560e-01,  1.42784371e-01,  1.68876201e-01,
         2.84500844e-01,  1.62625961e+00, -8.36649260e-01,
        -9.98500539e-01,  4.84319174e-01, -6.77003200e-01],
       [ 3.63958290e-01,  3.26326807e-01,  7.03145068e-01,
        -7.13778644e-01,  1.07370841e+00,  4.82633172e-01,
         4.77498943e-01, -1.32975879e+00, -6.86193316e-02,
        -7.35448152e-01, -1.35447914e+00, -8.06829025e-01,
         3.41737757e-01,  4.96641291e-02,  9.40576311e-02,
         1.37214061e+00,  6.93854620e-01, -2.00860931e+00,
         9.80549967e-01, -1.19401460e+00,  1.47709789e+00],
       [ 3.63958290e-01,  1.18293467e+00,  9.31757027e-01,
        -8.78587347e-01,  1.23870131e+00,  7.85179340e-01,
        -9.22275877e-01,  5.27775662e-01, -1.86734575e

In [45]:
test_y = test_df1['POSTSEASON'].values
test_y[0:5]
# y_v = np.where(test_y == 'F4', 1, 0)
test_yb = np.where(test_y == 'F4', 1, 0)
test_y[0:5],test_yb[0:5]

(array(['E8', 'E8', 'E8', 'E8', 'F4'], dtype=object), array([0, 0, 0, 0, 1]))

In [ ]:
test_y.shape,test_X.shape,test_yb.shape

In [53]:
Report_data = pd.DataFrame(columns=['Algorithm','Accuracy','Jaccard','F1-score','LogLoss'])

KNN


In [54]:
neigh = KNeighborsClassifier(n_neighbors = 5).fit(X_train,y_train) #training with the final parameters
yhat=neigh.predict(test_X)
mean_acc = metrics.accuracy_score(test_y, yhat)

f1score=metrics.f1_score(test_y, yhat,average='micro')
jaccard=jaccard_index(yhat, test_y)
LogLoss='NA'
Report_data = Report_data._append({"Algorithm":'KNN', "Accuracy":mean_acc,"Jaccard":jaccard, "F1-score":f1score,"LogLoss":LogLoss}, ignore_index=True)
Report_data

,Algorithm,Accuracy,Jaccard,F1-score,LogLoss
0,KNN,0.628571,0.458333,0.628571,NA


Decision Tree


In [55]:
dtree = DecisionTreeClassifier(criterion="entropy", max_depth = 1).fit(X_train,y_train)
yhat=dtree.predict(test_X)
mean_acc = metrics.accuracy_score(test_y, yhat)
f1score=metrics.f1_score(test_y, yhat,average='micro')
jaccard=jaccard_index(yhat, test_y)
LogLoss='NA'
Report_data = Report_data._append({"Algorithm":'Decision Tree', "Accuracy":mean_acc,"Jaccard":jaccard, "F1-score":f1score,"LogLoss":LogLoss}, ignore_index=True)
Report_data

,Algorithm,Accuracy,Jaccard,F1-score,LogLoss
0,KNN,0.628571,0.458333,0.628571,NA
1,Decision Tree,0.642857,0.473684,0.642857,NA


SVM


In [56]:
clf = svm.SVC(kernel='poly')
clf.fit(X_train, y_train) #y_t is y_train transformed earlier to binary as above
yhat = clf.predict(test_X)

mean_acc = metrics.accuracy_score(test_y, yhat)
f1score=metrics.f1_score(test_y, yhat,average='micro')
jaccard=jaccard_index(yhat, test_y)
LogLoss='NA'
Report_data = Report_data._append({"Algorithm":'SVM', "Accuracy":mean_acc,"Jaccard":jaccard, "F1-score":f1score,"LogLoss":LogLoss}, ignore_index=True)
Report_data

,Algorithm,Accuracy,Jaccard,F1-score,LogLoss
0,KNN,0.628571,0.458333,0.628571,NA
1,Decision Tree,0.642857,0.473684,0.642857,NA
2,SVM,0.685714,0.521739,0.685714,NA


Logistic Regression


In [57]:
#solver = sag gave a much better logloss value
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train, y_train)
yhat = LR.predict(test_X)

yhat_prob = LR.predict_proba(test_X)
#yhat_yhat_prob=list(zip(yhat,yhat_prob))

mean_acc = metrics.accuracy_score(test_y, yhat)
f1score=metrics.f1_score(test_y, yhat,average='micro')
jaccard=jaccard_index(yhat, test_y)
LogLoss=log_loss(test_y, yhat_prob)

Report_data = Report_data._append({"Algorithm":'LogisticRegression', "Accuracy":mean_acc,"Jaccard":jaccard, "F1-score":f1score,"LogLoss":LogLoss}, ignore_index=True)
Report_data

,Algorithm,Accuracy,Jaccard,F1-score,LogLoss
0,KNN,0.628571,0.458333,0.628571,NA
1,Decision Tree,0.642857,0.473684,0.642857,NA
2,SVM,0.685714,0.521739,0.685714,NA
3,LogisticRegression,0.685714,0.521739,0.685714,1.037187


# Report
You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Accuracy | Jaccard  | F1-score  | LogLoss |
|--------------------|----------|----------|-----------|---------|
| KNN                |     ?    |     ?    |     ?     | NA      |
| Decision Tree      |     ?    |     ?    |     ?     | NA      |
| SVM                |     ?    |     ?    |     ?     | NA      |
| LogisticRegression |     ?    |     ?    |     ?     |     ?   |


Something to keep in mind when creating models to predict the results of basketball tournaments or sports in general is that is quite hard due to so many factors influencing the game. Even in sports betting an accuracy of 55% and over is considered good as it indicates profits.


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">Watson Studio</a>



### Thank you for completing this lab!


## Author

Saeed Aghabozorgi


### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">Joseph Santarcangelo</a>




## Change Log


|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
|2021-04-03   | 2.1  | Malika Singla| Updated the Report accuracy |
| 2020-08-27  | 2.0  | Lavanya  |  Moved lab to course repo in GitLab |
|   |   |   |   |
|   |   |   |   |


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
